In [ ]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install textract
!pip install pandas
!pip install doc2txt
!pip install PyPDF2
!pip install PyMuPDF
!pip install python-docx
!pip install python-pptx
!pip install pytesseract
!pip install Pillow
!pip install streamlit
!pip install langchain
!pip install huggingface-hub
#!pip install paddlepaddle-gpu==2.5.2.post102 -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html
!pip install chromadb
!pip install gdown==4.6.0
!pip install requests
!pip install beautifulsoup4
!pip install sentence-transformers
!pip install -U langchain-community
!pip install spacy
!python -m spacy download en_core_web_sm

ERROR: Could not find a version that satisfies the requirement doc2txt (from versions: none)
ERROR: No matching distribution found for doc2txt
  Using cached PyMuPDF-1.24.7-cp310-none-manylinux2014_x86_64.whl (3.5 MB)
  Using cached PyMuPDFb-1.24.6-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (15.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import docx2txt
from PIL import Image
import pytesseract
import PyPDF2
import requests
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import text_splitter
from langchain.text_splitter import CharacterTextSplitter
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pptx
import csv
import os
import gdown
import chromadb
from chromadb import Client
from chromadb.config import Settings
import spacy


In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
# chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")

try:
    collection = chroma_client.create_collection("Company_Documents")
except Exception as e:
    print(f"Error creating collection: {e}")
    collection = chroma_client.get_collection("Company_Documents")


Error creating collection: Collection Company_Documents already exists


In [ ]:
nlp = spacy.load('en_core_web_sm')

def anonymize_names(text):
    doc = nlp(text)
    anonymized_text = text
    for entity in doc.ents:
        if entity.label_ == 'PERSON':
            anonymized_text = anonymized_text.replace(entity.text, 'ANONYMIZED')
    return anonymized_text

In [ ]:
# Define the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to download files from Google Drive using gdown
def download_files_from_drive(folder_id, local_folder):
    os.makedirs(local_folder, exist_ok=True)
    gdown.download_folder(id=folder_id, output=local_folder, quiet=False, use_cookies=False)
    upload_files_in_directory(local_folder)

def extract_content_from_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.pdf':
        return extract_content_from_pdf(file_path)
    elif file_extension == '.xlsx':
        return extract_content_from_xlsx(file_path)
    elif file_extension == '.csv':
        return extract_content_from_csv(file_path)
    elif file_extension == '.docx':
        return extract_content_from_docx(file_path)
    elif file_extension == '.txt':
        return extract_content_from_txt(file_path)
    elif file_extension in ['.jpg', '.jpeg', '.png']:
        return extract_content_from_image(file_path)
    elif file_extension == '.pptx':
        return extract_content_from_pptx(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

def extract_content_from_pdf(file_path):
    content = ""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            content += page.extract_text()
    return content

def extract_content_from_xlsx(file_path):
    df = pd.read_excel(file_path)
    return df.to_csv(index=False)

def extract_content_from_csv(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def extract_content_from_docx(file_path):
    return docx2txt.process(file_path)

def extract_content_from_txt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def extract_content_from_image(file_path):
    image = Image.open(file_path)
    return pytesseract.image_to_string(image)

def extract_content_from_pptx(file_path):
    prs = pptx.Presentation(file_path)
    content = ""
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                content += shape.text + "\n"
    return content

def extract_links_from_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    links = []
    for a_tag in soup.find_all('a', href=True):
        links.append(a_tag['href'])
    return links

def extract_content_from_url(url):
    try:
        response = requests.get(url)
        content_type = response.headers['Content-Type'].lower()
        if 'pdf' in content_type:
            return extract_content_from_pdf(url)
        elif 'image' in content_type:
            return extract_content_from_image(url)
        else:
            return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""

def process_links(text):
    links = extract_links_from_text(text)
    linked_content = ""
    for link in links:
        linked_content += extract_content_from_url(link)
    return linked_content

def chunk_and_embed_text(text):
    # Initialize the text splitter
    text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)

    # Split text into chunks
    chunks = text_splitter.split_text(text)

    # Embed each chunk
    embeddings = embedding_model.encode(chunks)

    # Convert embeddings from ndarray to list
    embeddings = [embedding.tolist() for embedding in embeddings]

    return chunks, embeddings

def upload_file_to_db(file_path, folder_name, access_role):
    content = extract_content_from_file(file_path)

    #Testing name anonymization
    if 'Meeting Transcripts' in file_path:
      content = anonymize_names(content)

    linked_content = process_links(content)
    full_content = content + linked_content
    chunks, embeddings = chunk_and_embed_text(full_content)
    document_id = os.path.basename(file_path)

    # Tag the document with metadata
    metadata = {
        'folder_name': folder_name,
        'file_type': os.path.splitext(file_path)[1].lower(),
        'access_role': access_role
    }

    # Add document and its embeddings to the collection if not already present
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        unique_id = f"{document_id}_{i}"
        existing_ids = collection.get(ids=[unique_id])
        if not existing_ids['ids']:
            collection.add(ids=[unique_id], documents=[chunk], embeddings=[embedding], metadatas=[metadata])
        else:
            print(f"Skipping existing ID: {unique_id}")

def upload_files_in_directory(directory_path):
    for root, _, files in os.walk(directory_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            access_role = 'Executive Access' if 'Contratcs' in file_path or 'Business Data' in file_path else 'General Access'
            try:
                upload_file_to_db(file_path, folder_name, access_role)
                print(f"Uploaded: {file_path} with role: {access_role}")
            except ValueError as e:
                print(e)








# Replace with the ID of your Google Drive folder
drive_folder_id = '1bFgwH3rvmZ6BD_Rl9sfMjB2MDtkZbjIy'
local_download_folder = 'downloaded_files'

# Download and process files
download_files_from_drive(drive_folder_id, local_download_folder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Retrieving folder list


Retrieving folder 18VnZcTlqN0IDuF1T1ltDGZjMlcb2jPzI Business Data
Processing file 1Yif4iluNpR4yckrz9cwY-5sjWTGVtU1K Alteryx - 24 Best Practices.pdf
Processing file 1LYqImnOKa0uCjgMjXo5Ozei8Zm8WLGvO Alteryx US Business Insights Q2 2022 Release Docs.pdf
Processing file 1r5kYFRdmrx3beK8DPvr8nHQFCMajJd5- alteryx-promote-datasheet.pdf
Processing file 1o5_ifRlkwaUXY4m_I3Pz5Yf4D212nGEe alteryxuipath-solution-briefv2-final.pdf
Processing file 12efYZkPA-R3cX0rKrkw55XznTf5TbNR6 Regional KPI Report - North America.pptx
Processing file 1kddPEFb4WHqLwhWGgicrnNUwsMZjGAaL rsm-alteryx-services.pdf
Retrieving folder 1UWg2Lt9buX6vHom_AeMiwgLuaMmKKGyf Contratcs
Processing file 1I95qFGNBw_ZrT6p13bm_Xx4vWt7KpLR5 Attach_rates.csv
Processing file 1hPUiLfcDwaVOUU0OunRqScR9canblFZ_ H2_renewal_v1.csv
Retrieving folder 1A4GA4DAJvBG023rIE-mDSafIXTdtNsjk Customer Report
Processing file 11fFMkzNzXqIA0v3Yf4x1aP9vhdX6YAdl Alteryx_Smart_Foods.pdf
Processing file 1jrJo349HGHsGkCJZ_bhsU0RNhruB--u9 Eco_Energy_Solutions.p

Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1Yif4iluNpR4yckrz9cwY-5sjWTGVtU1K
To: /content/downloaded_files/Business Data/Alteryx - 24 Best Practices.pdf
100%|██████████| 1.49M/1.49M [00:00<00:00, 41.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LYqImnOKa0uCjgMjXo5Ozei8Zm8WLGvO
To: /content/downloaded_files/Business Data/Alteryx US Business Insights Q2 2022 Release Docs.pdf
100%|██████████| 540k/540k [00:00<00:00, 78.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1r5kYFRdmrx3beK8DPvr8nHQFCMajJd5-
To: /content/downloaded_files/Business Data/alteryx-promote-datasheet.pdf
100%|██████████| 596k/596k [00:00<00:00, 95.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1o5_ifRlkwaUXY4m_I3Pz5Yf4D212nGEe
To: /content/downloaded_files/Business Data/alteryxuipath-solution-briefv2-final.pdf
100%|██████████| 333k/333k [00:00<00:00, 44.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=12e

Uploaded: downloaded_files/Web.txt with role: General Access
Uploaded: downloaded_files/Business Data/rsm-alteryx-services.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/Alteryx - 24 Best Practices.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/Alteryx US Business Insights Q2 2022 Release Docs.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/alteryxuipath-solution-briefv2-final.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/alteryx-promote-datasheet.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/Regional KPI Report - North America.pptx with role: Executive Access
Uploaded: downloaded_files/Customer Report/Fashion_Trends_Co..pdf with role: General Access
Uploaded: downloaded_files/Customer Report/Alteryx_Smart_Foods.pdf with role: General Access
Uploaded: downloaded_files/Customer Report/Tech_Gadgets_Ltd..pdf with role: General Access
Uploaded: downloaded_files